In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Make sure that caffe is on the python path:
#caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
import os
#print caffe_root
#sys.path.insert(0, caffe_root + 'python')
sys.path.append('/home/young/caffe-master/python/')
sys.path.append('/usr/lib/python2.7/dist-packages/')

import caffe

#rootdir = os.getcwd()
#print rootdir

print 'Done'

Done


In [8]:
# define unpickle file to read cifar10 data from python pickle file
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

In [11]:
# get mean ndarray from mean.binaryproto file by convert the file to .npy file

blob = caffe.proto.caffe_pb2.BlobProto()
data = open( '/home/young/Desktop/hashmodel/Zcifar10_hash/dataset/mean.binaryproto', 'rb' ).read()
blob.ParseFromString(data)
arr = np.array( caffe.io.blobproto_to_array(blob) )
mean = arr[0]

print mean.shape
#channel_swap = [2,1,0]

(3, 32, 32)


In [28]:

rootdir = '/home/young/Desktop/hashmodel'
rootdata = rootdir + '/Zcifar10_hash/dataset/'
TEST_DATA_FILE = rootdata + 'test_batch'
test_num = 10000
test  = unpickle(TEST_DATA_FILE)
test_data = test['data']
test_labels = test['labels']
test_caffe_in = test_data.reshape(test_num,  3, 32, 32)
test_caffe_in = np.array(test_caffe_in, np.float32) ##!!!!!!!!!!!!!!!!!!!!!!!!!!!!

for k in range(test_num):
    test_caffe_in[k][0] = test_caffe_in[k][0] - mean[0]
    test_caffe_in[k][1] = test_caffe_in[k][1] - mean[1]
    test_caffe_in[k][2] = test_caffe_in[k][2] - mean[2]


train_num = 50000
train_caffe_in = np.ndarray((train_num, 3, 32, 32), dtype=np.uint8)
train_labels = np.ndarray((train_num, ), dtype=np.uint8)

for i in xrange(5):
    TRAIN_DATA_FILE = rootdata + 'data_batch_' + str(i+1)
    num = 10000
    train_i  = unpickle(TRAIN_DATA_FILE)
    train_i_data = train_i['data']
    train_i_label = train_i['labels']
    train_i_nd = train_i_data.reshape(num,  3, 32, 32)
    train_caffe_in[i*num:(i+1)*num, :, :, :] = train_i_nd
    train_labels[i*num:(i+1)*num] = train_i_label

train_caffe_in = np.array(train_caffe_in, np.float32)
for k in range(train_num):
    train_caffe_in[k][0] = train_caffe_in[k][0] - mean[0]
    train_caffe_in[k][1] = train_caffe_in[k][1] - mean[1]
    train_caffe_in[k][2] = train_caffe_in[k][2] - mean[2]
    
# data processing needed here, centre, chanel change
rootmodel = rootdir + '/cifar10-32/'
net = caffe.Net(rootmodel + 'cifar10_32.prototxt', \
                rootmodel + 'cifar10_full_hash32_iter_70000.caffemodel')
net.set_mode_gpu() # use gpu model

#########################################################
#net.set_channel_swap(net.inputs[0], channel_swap)
#net.set_mean(net.inputs[0], mean)
#########################################################

test_out = net.forward_all(data = train_caffe_in)

test_prob = test_out['prob']

tmp = 0
for i in xrange(train_num):
    if (test_prob[i].argmax() == train_labels[i]):
        tmp = tmp+1
print tmp*100.0/train_num
print 'Done'

97.096
Done


In [29]:
# iterate all the bits from 16~512
from scipy.io import savemat

rootdir = '/home/young/Desktop/hashmodel'

netfile = ['16', '24', '32', '48', '64', '128', '256', '512']
modelfile = ['cifar10_full_hash16_iter_70000.caffemodel', 'cifar10_full_hash24_iter_70000.caffemodel', \
             'cifar10_full_hash32_iter_70000.caffemodel', 'cifar10_full_hash48_iter_70000.caffemodel', \
             'cifar10_full_hash64_iter_70000.caffemodel', 'cifar10_full_hash128_iter_70000.caffemodel', \
             'cifar10_full_hash256_iter_70000.caffemodel', 'cifar10_full_feature512_iter_70000.caffemodel']


roothash = rootdir + '/Zcifar10_hash/hash/'
for i in xrange(8):
    print 'processing %s bit hashing' % netfile[i]
    rootmodel = rootdir + '/cifar10-' + netfile[i] + '/'
    net = caffe.Net(rootmodel + 'cifar10_' + netfile[i] + '.prototxt', \
                    rootmodel + modelfile[i])
    net.set_mode_gpu() # use gpu model
    test_out = net.forward_all(data = test_caffe_in)
    train_out = net.forward_all(data = train_caffe_in)
    #hash = np.vstack((test_out.get('hash'+netfile[i]), train_out.get('hash'+netfile[i])))
    #savemat(roothash+'hash'+netfile[i]+'.mat', {'hash':hash})
    test_hash = test_out.get('hash'+netfile[i])
    train_hash = train_out.get('hash'+netfile[i])
    savemat(roothash+'hash'+netfile[i]+'.mat', {'testh':test_hash, 'trainh':train_hash})
"""    
label = np.ndarray((test_num+train_num,), dtype = np.uint8)
label[0:test_num] = test_labels
label[test_num:test_num+train_num] = train_labels
savemat(roothash+'label.mat', {'label':label})
"""
savemat(roothash+'label.mat', {'test_label':test_labels, 'train_label':train_labels})


print 'Done'

processing 16 bit hashing
processing 24 bit hashing
processing 32 bit hashing
processing 48 bit hashing
processing 64 bit hashing
processing 128 bit hashing
processing 256 bit hashing
processing 512 bit hashing
Done
